## Cодержание:
* [First Bullet Header](#first-bullet)
* [Second Bullet Header](#second-bullet)

# Команда: Бета Банк 


**Цель:** Создать CLTV модель, которая будет выдавать вероятности перехода в каждый из 17 продуктовых кластеров в течение 12 месяцев.

Альфа-Банком предоставлены следующие **данные**, описание из файла **feature_description.xlsx**:

-   **`train_data.pqt`и `test_data.pqt` – данные о клиентах за 3 месяца:**
   
    Возможно тут описание длатасета
    - `st_id` – захэшированное id магазина;
    - `pr_sku_id` – захэшированное id товара;
    - `date` – дата;
    - `pr_sales_type_id` – флаг наличия промо;
    - `pr_sales_in_units` – число проданных товаров без признака промо;
    - `pr_promo_sales_in_units` – число проданных товаров с признаком промо;
    - `pr_sales_in_rub` – продажи без признака промо в РУБ;
    - `pr_promo_sales_in_rub` – продажи с признаком промо в РУБ;


  
Метрикой качества выступает **ROC-AUC**.

Данные о клиентах и масскированы.

## Библиотеки

In [ ]:
# Необходимые библиотеки
from sklearn.ensemble import StackingClassifier, RandomForestClassifier, ExtraTreesClassifier
import warnings
from IPython.display import display, HTML


import os
from sklearn.utils import resample
import seaborn as sns
import matplotlib.pyplot as plt

from catboost import CatBoostClassifier
# import optuna
from catboost import CatBoostClassifier, Pool
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder



import numpy as np
import time
import json

import pandas as pd
pd.set_option('display.float_format', '{:.4f}'.format)
pd.set_option('display.max_rows', 93)

# Отключить все предупреждения временно
import warnings
warnings.filterwarnings("ignore")

## Задание 

1. Качественно оформите код модели
2. Доработка решения на платформе будет открыта до 18 апреля 12:00
3. Обязательно наличие .README;
4. Код должен быть читабелен и понятен;
5. Решение должно быть воспроизводимо: эксперты должны иметь возможность протестировать ваше решение на финале.

## Загрузка и изучение данных

In [ ]:
def read_df(path: str) -> pd.DataFrame:
    """
    Функция для чтения DataFrame из Parquet-файла.

    Параметры:
    path (str): Путь к Parquet-файлу.

    Возвращает:
    pd.DataFrame: DataFrame, прочитанный из Parquet-файла.

    """
    if os.path.exists(path):
        df = pd.read_parquet(path)
        print(f'Успешно: Данные {path} загружены')
        return df
    else:
        print(f'Ошибка: {path} не найден')
        return None

In [ ]:
# Путь до файла train_df
path_train_df = "/kaggle/input/df-restore-cal-avg-start-cluster-3-pqt/train_data.pqt"

# Путь до файла test_df
path_test_df = "/kaggle/input/df-restore-cal-avg-start-cluster-3-pqt/test_data.pqt"

train_df = read_df(path=path_train_df)
test_df = read_df(path=path_test_df)

## Предобработка данных

### Объединение датасетов


In [ ]:
df = pd.concat([train_df, test_df], ignore_index=True)

## Feature engineering

Тут кратко описание секции

###  AVG

Очень много плохих столбцов sum но  cnt хорошие и можно сгенирировать avg

In [ ]:
df['avg_a_oper_1m'] = df['sum_a_oper_1m'] / df['cnt_a_oper_1m']
df['avg_b_oper_1m'] = df['sum_b_oper_1m'] / df['cnt_b_oper_1m']
df['avg_c_oper_1m'] = df['sum_c_oper_1m'] / df['cnt_c_oper_1m']

df['avg_deb_d_oper_1m'] = df['sum_deb_d_oper_1m'] / df['cnt_deb_d_oper_1m']
df['avg_cred_d_oper_1m'] = df['sum_cred_d_oper_1m'] / df['cnt_cred_d_oper_1m']

df['avg_deb_e_oper_1m'] = df['sum_deb_e_oper_1m'] / df['cnt_deb_e_oper_1m']
df['avg_cred_e_oper_1m'] = df['sum_cred_e_oper_1m'] / df['cnt_cred_e_oper_1m']


df['avg_deb_f_oper_1m'] = df['sum_deb_f_oper_1m'] / df['cnt_deb_f_oper_1m']
df['avg_cred_f_oper_1m'] = df['sum_cred_f_oper_1m'] / df['cnt_cred_f_oper_1m']

df['avg_deb_g_oper_1m'] = df['sum_deb_g_oper_1m'] / df['cnt_deb_g_oper_1m']
df['avg_cred_g_oper_1m'] = df['sum_cred_g_oper_1m'] / df['cnt_cred_g_oper_1m']

df['avg_deb_h_oper_1m'] = df['sum_deb_h_oper_1m'] / df['cnt_deb_h_oper_1m']
df['avg_cred_h_oper_1m'] = df['sum_cred_h_oper_1m'] / df['cnt_cred_h_oper_1m']


df['avg_a_oper_3m'] = df['sum_a_oper_3m'] / df['cnt_a_oper_3m']
df['avg_b_oper_3m'] = df['sum_b_oper_3m'] / df['cnt_b_oper_3m']
df['avg_c_oper_3m'] = df['sum_c_oper_3m'] / df['cnt_c_oper_3m']

df['avg_deb_d_oper_3m'] = df['sum_deb_d_oper_3m'] / df['cnt_deb_d_oper_3m']
df['avg_cred_d_oper_3m'] = df['sum_cred_d_oper_3m'] / df['cnt_cred_d_oper_3m']

df['avg_deb_e_oper_3m'] = df['sum_deb_e_oper_3m'] / df['cnt_deb_e_oper_3m']
df['avg_cred_e_oper_3m'] = df['sum_cred_e_oper_3m'] / df['cnt_cred_e_oper_3m']

df['avg_deb_f_oper_3m'] = df['sum_deb_f_oper_3m'] / df['cnt_deb_f_oper_3m']
df['avg_cred_f_oper_3m'] = df['sum_cred_f_oper_3m'] / df['cnt_cred_f_oper_3m']

df['avg_deb_g_oper_3m'] = df['sum_deb_g_oper_3m'] / df['cnt_deb_g_oper_3m']
df['avg_cred_g_oper_3m'] = df['sum_cred_g_oper_3m'] / df['cnt_cred_g_oper_3m']

df['avg_deb_h_oper_3m'] = df['sum_deb_h_oper_3m'] / df['cnt_deb_h_oper_3m']
df['avg_cred_h_oper_3m'] = df['sum_cred_h_oper_3m'] / df['cnt_cred_h_oper_3m']

### Удаление плохих столбцов

Буду пробовать не удалять

In [ ]:
# columns_to_drop = [
#     'balance_amt_max',
#     'balance_amt_min',
#     'balance_amt_day_avg',
#     'index_city_code',
#     'max_founderpres',
#     'min_founderpres',
#     'ogrn_exist_months',
#     'sum_a_oper_1m',
#     'sum_b_oper_1m',
#     'sum_c_oper_1m',
#     'sum_deb_d_oper_1m',
#     'sum_cred_d_oper_1m',
#     'sum_deb_e_oper_1m',
#     'sum_cred_e_oper_1m',
#     'sum_deb_f_oper_1m',
#     'sum_cred_f_oper_1m',
#     'sum_deb_g_oper_1m',
#     'sum_cred_g_oper_1m',
#     'sum_deb_h_oper_1m',
#     'sum_cred_h_oper_1m',
#     'sum_a_oper_3m',
#     'sum_b_oper_3m',
#     'sum_c_oper_3m',
#     'sum_deb_d_oper_3m',
#     'sum_cred_d_oper_3m',
#     'sum_deb_e_oper_3m',
#     'sum_cred_e_oper_3m',
#     'sum_deb_f_oper_3m',
#     'sum_cred_f_oper_3m',
#     'sum_deb_g_oper_3m',
#     'sum_cred_g_oper_3m',
#     'sum_deb_h_oper_3m',
#     'sum_cred_h_oper_3m']


# df = df.drop(columns=columns_to_drop)

### Восстановление категориальных данных

In [ ]:
def restore_cal(x):
    if x.isna().any() and not x.isna().all():
      return x.fillna(x.dropna().iloc[-1])
    return x

In [ ]:
import time 

cat_columns_to_restore = ['channel_code', 'city',
                          'city_type', 'ogrn_month', 'ogrn_year', 'okved', 'segment']

for column in cat_columns_to_restore:
  df[column] = df.groupby('id')[column].apply(
      lambda x: restore_cal(x)).reset_index()[column]
  print(f"Колонка - {column} - восстановлена")

In [ ]:
df.to_parquet("df.pqt")

In [ ]:
df = pd.read_parquet("/kaggle/working/df.pqt")

In [ ]:
df = pd.read_parquet("df.pqt")
cat_cols = [
          "channel_code", "city", "city_type",
          "okved", "segment", "start_cluster", "ogrn_month", "ogrn_year",
      ]



df['date'] = df['date'].replace({'month_4': 'month_1', 'month_5': 'month_2', 'month_6': 'month_3'})

df[cat_cols] = df[cat_cols].astype("object")

### Создание таблицы с 3 месяцами 

In [ ]:
df

In [ ]:
cat_cols = [
    "channel_code", "city", "city_type",
    "okved", "segment", "ogrn_month", "ogrn_year",
]

cat_cols_month_1 = [f'{col}_month_1' for col in cat_cols]
cat_cols_month_2 = [f'{col}_month_2' for col in cat_cols]



pivot_df = df.pivot_table(index='id', columns='date', aggfunc='first')

pivot_df.columns = [f'{col[0]}_{col[1]}' for col in pivot_df.columns]

pivot_df.reset_index(inplace=True)
pivot_df = pivot_df.drop(
    columns=['end_cluster_month_1', 'end_cluster_month_2'] + cat_cols_month_1 + cat_cols_month_2, axis=0)

categorical_columns = pivot_df.select_dtypes(include=['object']).columns
pivot_df[categorical_columns] = pivot_df[categorical_columns].fillna("missing")

In [ ]:
pivot_df[['start_cluster_month_1', 'start_cluster_month_2', 'start_cluster_month_3']]

In [ ]:
 len(list(pivot_df.select_dtypes(include=['number']).columns[1:]))

In [ ]:
numeric_cols = pivot_df.select_dtypes(include=['number']).columns[1:]

for col in numeric_cols:
    print(col)

In [ ]:
numeric_cols = pivot_df.select_dtypes(include=['number']).columns[1:]

for col in numeric_cols:
    pivot_df[f'{col}_diff_2_1'] = pivot_df[f'{col}_month_2'] - pivot_df[f'{col}_month_1']

for col in numeric_cols:
    pivot_df[f'{col}_diff_3_2'] = pivot_df[f'{col}_month_3'] - pivot_df[f'{col}_month_2']

In [ ]:
df = pivot_df

### Воостановление start_claster

In [ ]:
train_data = df[df['start_cluster_month_3'] != 'missing'].drop(
    ['id',  'end_cluster_month_3'], axis=1)
predict_data = df[df['start_cluster_month_3'] == 'missing'].drop(
    ['id', 'end_cluster_month_3'], axis=1)

X = train_data.drop('start_cluster_month_3', axis=1)
y = train_data['start_cluster_month_3']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# # Определение желаемого количества экземпляров каждого класса
# desired_class_count = 750  # Укажите ваше желаемое количество экземпляров

# # Обработка дисбаланса классов
# balanced_data = pd.DataFrame()
# for cluster in train_data['start_cluster_month_3'].unique():
#     cluster_data = train_data[train_data['start_cluster_month_3'] == cluster]
#     if len(cluster_data) < desired_class_count:
#         resampled_data = resample(
#             cluster_data, replace=True, n_samples=desired_class_count, random_state=42)
#     else:
#         resampled_data = cluster_data.sample(
#             n=desired_class_count, replace=False, random_state=42)
#     balanced_data = pd.concat([balanced_data, resampled_data])
# display(balanced_data['start_cluster_month_3'].value_counts())



# X = balanced_data.drop('start_cluster_month_3', axis=1)
# y = balanced_data['start_cluster_month_3']

# categorical_columns = X.select_dtypes(include=['object']).columns
# X[categorical_columns] = X[categorical_columns].fillna("missing")


In [ ]:
catboost_model_start_cluster = CatBoostClassifier(iterations=1024,
                           depth=8,
                           learning_rate=0.075,
                           random_seed=47,
                           loss_function='MultiClass',
                           task_type="GPU",
                           devices='0',
                           early_stopping_rounds=20
                           )

In [ ]:
def train_catboost(model, x_train, y_train, x_val, y_val, cat_names):

    model.fit(
        x_train, y_train,
        cat_features=np.array(cat_names),
        eval_set=(x_val, y_val),
        verbose=100  # через сколько итераций выводить стату
    )
    model.save_model('catboost_model_start_cluster.json')  # сохранение модели
    feature_importance = model.get_feature_importance(
        prettified=True)  # датасет с важностью признаков

    return feature_importance

In [ ]:
cat_names = X.select_dtypes(include=['object']).columns

feature_importance = train_catboost(
    catboost_model_start_cluster, X, y, X_val, y_val, cat_names)

In [ ]:
X_predict = predict_data.drop('start_cluster_month_3', axis=1)
predicted_clusters = catboost_model_start_cluster.predict(X_predict)

In [ ]:
predicted_clusters_flat = np.ravel(predicted_clusters)
class_counts = pd.Series(predicted_clusters_flat).value_counts()
print(class_counts)

In [ ]:
predicted_index = 0

df_restore_start_cluster = df.copy()
for index, row in df_restore_start_cluster.iterrows():
    # Проверяем, содержится ли в столбце 'date' значение 'month6' и id >= 100000
    if row['id'] >= 200000:
        # Вставляем значение из серии в столбец 'start_cluster_month_3' текущей строки
        df_restore_start_cluster.at[index,
                                    'start_cluster_month_3'] = predicted_clusters[predicted_index][0]
        # Увеличиваем индекс текущей строки в серии
        predicted_index += 1

In [ ]:
matching_rows = df_restore_start_cluster[df_restore_start_cluster['id'] >= 200000].loc[(df_restore_start_cluster['start_cluster_month_1'] == df_restore_start_cluster['start_cluster_month_2']) & (
    df_restore_start_cluster['start_cluster_month_2'] == df_restore_start_cluster['start_cluster_month_3'])]
matching_rows

In [ ]:
df.to_parquet("df_0.902.pqt")

## Обучение модели <a class="anchor" id="first-bullet"></a>

In [ ]:
train_df = df_restore_start_cluster[df_restore_start_cluster['id']< 200000]
test_df = df_restore_start_cluster[df_restore_start_cluster['id'] >= 200000]

X = train_df.drop(["id"], axis=1) # оставляю end_cluster чтобы получить пропорцию классов, а потом ниже удалю в коде
y = train_df["end_cluster_month_3"]

x_train, x_val, y_train, y_val = train_test_split(X, y,
                                                  test_size=0.2,
                                                  random_state=42)

In [ ]:
x_train['end_cluster_month_3'].value_counts()

In [ ]:
y_train = x_train['end_cluster_month_3']
x_train = x_train.drop(['end_cluster_month_3'], axis=1)
x_val = x_val.drop(['end_cluster_month_3'], axis=1)

display(x_train.shape, y_train.shape, x_val.shape, y_val.shape)

In [ ]:
catboost_model_end_cluster = CatBoostClassifier(iterations=2000,
                           depth=6,
                           learning_rate=0.075,
                           random_seed=47,
                           loss_function='MultiClass',
                           task_type="GPU",
                           devices='0',
                           early_stopping_rounds=20
                          )


In [ ]:
def train_catboost(model, x_train, y_train, x_val, y_val, cat_names):

    model.fit(
    x_train, y_train,
    cat_features=np.array(cat_names),
    eval_set=(x_val, y_val),
    verbose=15 # через сколько итераций выводить стату
    )
    model.save_model('catboost_model_end_claster.json') # сохранение модели
    feature_importance = model.get_feature_importance(prettified=True) # датасет с важностью признаков

    return feature_importance

In [ ]:
cat_names = x_train.select_dtypes(include=['object']).columns



feature_importance = train_catboost(catboost_model_end_cluster, x_train, y_train, x_val, y_val, cat_names)

## Тестирование модели

In [ ]:
def weighted_roc_auc(y_true, y_pred, labels, weights_dict):
    unnorm_weights = np.array([weights_dict[label] for label in labels])
    weights = unnorm_weights / unnorm_weights.sum()
    classes_roc_auc = roc_auc_score(y_true, y_pred, labels=labels,
                                    multi_class="ovr", average=None)
    return sum(weights * classes_roc_auc)

In [ ]:
cluster_weights = pd.read_excel("/kaggle/input/df-restore-cal-avg-start-cluster-3-pqt/cluster_weights.xlsx").set_index("cluster")
weights_dict = cluster_weights["unnorm_weight"].to_dict()

In [ ]:
y_pred_proba = catboost_model_end_cluster.predict_proba(x_val)
weighted_roc_auc(y_val, y_pred_proba, catboost_model_end_cluster.classes_, weights_dict)

Прогноз на тестовой выборке

In [ ]:
sample_submission_df = pd.read_csv("/kaggle/input/df-restore-cal-avg-start-cluster-3-pqt/sample_submission.csv") # поменять на свой
last_m_test_df = test_df
last_m_test_df = last_m_test_df.drop(["id" , 'end_cluster_month_3'], axis=1)

pool2 = Pool(data=last_m_test_df, cat_features=np.array(cat_names))

test_pred_proba = catboost_model_end_cluster.predict_proba(pool2) # last_m_test_df
test_pred_proba_df = pd.DataFrame(test_pred_proba, columns=catboost_model_end_cluster.classes_)
sorted_classes = sorted(test_pred_proba_df.columns.to_list())
test_pred_proba_df = test_pred_proba_df[sorted_classes]

sample_submission_df[sorted_classes] = test_pred_proba_df
sample_submission_df.to_csv("catboost_1.csv", index=False) # сохранение модели

In [ ]:
sample_submission_df

---
## Выводы и резюме

Мы решали **задачу прогнозирования временного ряда спроса товаров** собственного производства на 14 дней вперёд. 

Заказчиком предоставлены исторические данные о **продажах за 1 год**, а также в закодированном виде товарная иерархия и информация о магазинах.  
Прогнозировалось **число проданных товаров в штуках  `pr_sales_in_units`** для каждого **SKU/товара** (2050 шт. в обучающей выборке) в каждом из **10 магазинов**.

Основные **закономерности**, выявленные в результате анализа: 
- ***Годовой тренд***  - спад средних продаж в зимний сезон октябрь-март.
- ***Недельная сезонность*** - пик продаж в субботу, спад в понедельник.
- В течение года несколько высоких ***пиков спроса, в основном в районе праздников***. Самые резкие подъёмы продаж в период Нового года и Пасхи. Подъем продаж начинается за несколько дней до.
- 40,6% записей относятся к продажам по промоакциям. Возможны одновременные продажи товара в одном магазине по промо и без. 
- В данных представлены продукты с ***неполными временными рядами***: продавались только в дни около Пасхи, начали продаваться полгода назад.
- Во всех магазинах разный ассортимент товаров даже при условии одинаковых характеристик торговой точки.
- Все мета-признаки как характеристики магазинов и товаров показали влияние на средний спрос

На основе имеющихся данных **сгенерированы новые признаки:**  
- Календарные: день недели, число месяца, номер недели, флаг выходного дня (взят из доп. таблицы)
- Лаговые признаки 1-30 дней
- Скользящее среднее за 7 и 14 предыдущих дней
- Кластеризация по характеристикам магазинов и товаров
    
Чтобы временные ряды каждой комбинации Магазин-Товар были полными создан новый датасет, в который добавлены отсутствующие даты с нулевыми продажами.

 Обучение, валидация и выбор лучшего набора гиперпараметров проводится на **кросс-валидации Walk Forward**: подбор гиперпараметров на фолде проводится на valid-выборке, оценка лучшей модели на фолде на test-выборке.   
В итоге выбрана одна модель среди лучших на каждом фолде.

 Предсказание спроса обученной моделью делается последовательно на каждый следующий день с промежуточным перерасчётом лаговых признаков (учитывается предсказанное значение спроса в предыдущий день).

 Для оценки модели использовалась метрика качества  **WAPE**, посчитанная на уровне Магазин-Товар-Дата.  
 
Лучший результат по качеству и скорости показала модель градиентного бустинга **LightGBM**.  <br>
Полученный результат: WAPE = **0,47**, превышает baseline (предсказание последним известным значением) с метрикой 69%.


